não ta funcionando ainda pera ai

In [7]:
import numpy as np
from svgpathtools import svg2paths
from PIL import Image, ImageDraw
import heapq
import random

def generate_occupancy_grid(width, height, obstacle_ratio=0.1):
    grid = np.ones((width, height), dtype=np.uint8) * 255
    num_obstacles = int(width * height * obstacle_ratio)
    for _ in range(num_obstacles):
        x = random.randint(0, width - 1)
        y = random.randint(0, height - 1)
        grid[x, y] = 0
    return grid

def save_grid_as_image(grid, filepath):
    image = Image.fromarray(grid)
    image.save(filepath)

width = 100
height = 100
obstacle_ratio = 0.1

grid = generate_occupancy_grid(width, height, obstacle_ratio)
save_grid_as_image(grid, 'occupancy_grid.png')

def load_image(filepath):
    image = Image.open(filepath).convert('L')  # Convert to grayscale
    image_array = np.array(image)
    return image_array

# Load the generated occupancy grid image
image_path = 'occupancy_grid.png'
image_array = load_image(image_path)

# Convert image array to grid format
grid = (image_array == 255).astype(int).tolist()  # Convert white (255) to 1 (free space), black (0) to 0 (obstacle)

# Define start and goal positions
start = (10, 10)
goal = (90, 85)

In [4]:
class Node:
    def __init__(self, parent, position):
        self.parent = parent
        self.position = position
        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position

    def __lt__(self, other):
        return self.f < other.f

In [5]:
def astar(grid, start, end, visited_nodes):
    """Returns a list of tuples as a path from the given start to the given end in the given grid"""

    # Create start and end node
    start_node = Node(None, start)
    end_node = Node(None, end)

    # Initialize both open and closed lists
    open_list = []
    closed_list = set()

    # Add the start node
    heapq.heappush(open_list, start_node)

    while open_list:
        # Get the current node
        current_node = heapq.heappop(open_list)
        closed_list.add(current_node.position)

        # Found the goal
        if current_node == end_node:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]  # Return reversed path

        # Generate children (adjacent squares)
        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0)]:  # 4 directions (no diagonals)

            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Make sure within range and walkable terrain
            if (0 <= node_position[0] < len(grid) and
                0 <= node_position[1] < len(grid[0]) and
                grid[node_position[0]][node_position[1]] == 1):
                new_node = Node(current_node, node_position)
                children.append(new_node)

        # Loop through children
        for child in children:
            if child.position in closed_list:
                continue

            # Create the f, g, and h values
            child.g = current_node.g + 1
            child.h = abs(child.position[0] - end_node.position[0]) + abs(child.position[1] - end_node.position[1])
            child.f = child.g + child.h

            # Avoid nodes that have been visited in previous paths but give them lower priority
            if child.position in visited_nodes:
                child.f += 10  # Penalize visited nodes to prioritize unvisited nodes

            # Child is already in the open list
            if any(open_node.position == child.position and child.g > open_node.g for open_node in open_list):
                continue

            # Add the child to the open list
            heapq.heappush(open_list, child)

    return None  # No path found

In [8]:
from tqdm import tqdm

def find_multiple_paths(grid, start, end, num_paths):
    all_paths = []
    visited_nodes = set()

    for _ in range(num_paths):
        path = astar(grid, start, end, visited_nodes)
        print("path found!")
        if path is None:
            break
        all_paths.append(path)
        visited_nodes.update(path)
    
    return all_paths

paths = find_multiple_paths(grid, start, goal, 3)
print(len(paths))
for path in paths:
    print(path)

path found!
path found!
path found!
3
[(10, 10), (11, 10), (11, 11), (12, 11), (13, 11), (13, 12), (13, 13), (13, 14), (14, 14), (14, 15), (14, 16), (15, 16), (15, 17), (16, 17), (16, 18), (17, 18), (17, 19), (18, 19), (19, 19), (20, 19), (21, 19), (21, 20), (22, 20), (22, 21), (22, 22), (23, 22), (23, 23), (24, 23), (24, 24), (24, 25), (25, 25), (26, 25), (27, 25), (28, 25), (29, 25), (30, 25), (31, 25), (32, 25), (33, 25), (34, 25), (35, 25), (35, 26), (36, 26), (36, 27), (36, 28), (36, 29), (37, 29), (37, 30), (37, 31), (37, 32), (38, 32), (38, 33), (38, 34), (39, 34), (39, 35), (39, 36), (39, 37), (40, 37), (41, 37), (42, 37), (43, 37), (44, 37), (45, 37), (46, 37), (46, 38), (47, 38), (48, 38), (49, 38), (50, 38), (51, 38), (52, 38), (53, 38), (54, 38), (54, 39), (55, 39), (56, 39), (57, 39), (58, 39), (59, 39), (60, 39), (61, 39), (62, 39), (63, 39), (64, 39), (64, 40), (64, 41), (65, 41), (65, 42), (65, 43), (66, 43), (67, 43), (67, 44), (67, 45), (68, 45), (68, 46), (68, 47), (

In [9]:
def draw_routes_on_map(image, routes, output_path):
    image = image.convert("RGB")  # Convert to RGB for coloring
    draw = ImageDraw.Draw(image)
    
    # Draw each route in a different color
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 165, 0)]
    for i, route in enumerate(routes):
        color = colors[i % len(colors)]
        for position in route:
            draw.point(position, fill=color)  # Draw route points in different colors
    
    image.save(output_path)
    
# Load the original image to draw the routes
original_image = Image.open(image_path)
draw_routes_on_map(original_image, paths, 'routes_on_map.png')